In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

data_potencia = pd.read_csv('Data\Potencia_compresores(in).csv', sep = ';') # Importar los datos de potencia
data_compresores = pd.read_csv('Data\data_alterada_dos_compresores.csv', sep = ';') # Importar los datos de compresores

data_potencia['datetime'] = pd.to_datetime(data_potencia['datetime'],) # Convertir la columna datetime a tipo datetime
data_potencia['datetime'] = data_potencia['datetime'].dt.tz_localize(None,) # Eliminar la zona horaria de la columna datetime
data_compresores['Date'] = pd.to_datetime(data_compresores['Date'],) # Convertir la columna Date a tipo datetime
data_compresores['Date'] = data_compresores['Date'].dt.tz_localize(None,)

# Procesamiento

### Corrección de las potencias

In [ ]:
columnas_a_corregir = ["potencia-compresor-CB1", "potencia-compresor-CB2"]

for col in columnas_a_corregir:
    data_potencia[col] = data_potencia[col].astype(str)  # Convertir a string para manejar la longitud
    data_potencia[col] = data_potencia[col].str.replace(r"\D", "", regex=True)  # Eliminar puntos/comas extrañas
    data_potencia[col] = data_potencia[col].apply(lambda x: float(x[:3] + "." + x[3:]) if x.isdigit() else x)
    data_potencia[col] = np.where(data_potencia[col] == '', np.nan, data_potencia[col])
    data_potencia[col] = data_potencia[col].astype(float)  # Convertir a float después de la limpieza
    data_potencia[col] = np.where(data_potencia[col] > 200, data_potencia[col] / 1000, data_potencia[col])  # Dividir por 1000 si es mayor a 200 debido a que es un error de lectura

In [ ]:
data_potencia['potencia-compresor-CB1'].plot()

### Corrección de Fechas

In [ ]:
data_compresores['Date'] = pd.to_datetime(data_compresores['Date'] + pd.Timedelta(hours=1)) # Se sabía que la fecha estaba en UTC y se le sumó 1 hora para corregir el desfase horario
data_potencia['datetime'] = pd.to_datetime(data_potencia['datetime'] + pd.Timedelta(hours=5)) # Se sabía que la fecha estaba en UTC y se le sumó 5 horas para corregir el desfase horario

### Merge de DataFrames

In [ ]:
compresores = data_compresores.pivot_table(index = 'Date', columns='name', values='altered_data') # Tabla dinámica

In [ ]:
compresores

In [ ]:
data_potencia.set_index('datetime', inplace=True)

In [ ]:
compresores = pd.merge_asof(compresores, data_potencia, left_on='Date', right_index=True)
# Combinar los dataframes se usa merge_asof para hacer una combinación de tipo "asof" que es útil para datos de series temporales. Esto significa que se combinarán las filas de ambos dataframes basándose en la fecha más cercana, pero no necesariamente en la misma fecha exacta. Esto es útil cuando los datos de los dos dataframes no están perfectamente alineados en el tiempo.

In [ ]:
compresores.drop(columns=['vpm-12-totalizer-mscf', 'vpm-14-totalizer-mscf', 'Unnamed: 0'], inplace=True) # Eliminar columnas innecesarias

In [ ]:
mapa = {
    'vpm-12-flow-scfm' : 'Flujo Compresor 2 (SCFM)',
    'vpm-12-pressure-psi': 'Presion Compresor 2 (PSI)',
    'vpm-12-temperature': 'Temperatura Compresor 2 (°C)',
    'potencia-compresor-CB2' : 'Potencia Compresor 2 (kW)',
    'vpm-14-flow-scfm' : 'Flujo Compresor 1 (SCFM)',
    'vpm-14-pressure-psi': 'Presion Compresor 1 (PSI)',
    'vpm-14-temperature': 'Temperatura Compresor 1 (°C)',
    'potencia-compresor-CB1' : 'Potencia Compresor 1 (kW)',
}# Cambiar los nombres de las columnas

compresores.rename(columns=mapa, inplace=True)
compresores.rename_axis('Fecha', inplace=True)

In [ ]:
compresores['KPI Compresor 1'] = compresores['Potencia Compresor 1 (kW)'] / compresores['Flujo Compresor 1 (SCFM)'] # Calcular el KPI del compresor 1
compresores['KPI Compresor 2'] = compresores['Potencia Compresor 2 (kW)'] / compresores['Flujo Compresor 2 (SCFM)'] # Calcular el KPI del compresor 2
compresores['KPI Compresor 1'] = compresores['KPI Compresor 1'].replace([np.inf, -np.inf], np.nan)  # Reemplazar inf por NaN
compresores['KPI Compresor 2'] = compresores['KPI Compresor 2'].replace([np.inf, -np.inf], np.nan)  # Reemplazar inf por NaN

In [ ]:
compresores # Dataframe final
compresores.to_csv('Data\compresores.csv', index = True) # Guardar el dataframe final como CSV

### Carga de todos los compresores

In [ ]:
from sqlalchemy import create_engine

# Configuración de la conexión a la base de datos
db_user = 'postgres'
db_password = '1234'
db_host = 'localhost'
db_port = '5432'
db_name = 'postgres'

# Crear la conexión
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}',)

# Subir el DataFrame a PostgreSQL
tabla = 'compresores'  # Nombre de la tabla en la base de datos
compresores.to_sql(tabla, engine, if_exists='append', index=True, index_label='Fecha', schema='public', method='multi', chunksize=1000)
print(f"Los datos se han subido correctamente a la tabla '{tabla}' en la base de datos '{db_name}'.")

# Separación de Tablas por Compresores

In [ ]:
compresores_horario = compresores.resample('H').mean()  # Muestreo Horario

In [ ]:
compresores_horario['KPI Compresor 1'] = compresores_horario['Potencia Compresor 1 (kW)'] / compresores_horario['Flujo Compresor 1 (SCFM)']
compresores_horario['KPI Compresor 2'] = compresores_horario['Potencia Compresor 2 (kW)'] / compresores_horario['Flujo Compresor 2 (SCFM)'] 
compresores_horario['KPI Compresor 1'] = compresores_horario['KPI Compresor 1'].replace([np.inf, -np.inf], np.nan)  # Reemplazar inf por NaN
compresores_horario['KPI Compresor 2'] = compresores_horario['KPI Compresor 2'].replace([np.inf, -np.inf], np.nan)  # Reemplazar inf por NaN

## Correlaciones Minutales

In [ ]:
# Filtrado de Datos

compresor_1 = compresores[['Flujo Compresor 1 (SCFM)', 'Potencia Compresor 1 (kW)', 'KPI Compresor 1', 'Presion Compresor 1 (PSI)', 'Temperatura Compresor 1 (°C)']].dropna()
compresor_2 = compresores[['Flujo Compresor 2 (SCFM)', 'Potencia Compresor 2 (kW)', 'KPI Compresor 2', 'Presion Compresor 2 (PSI)', 'Temperatura Compresor 2 (°C)']].dropna()
compresor_1 = compresor_1[compresor_1['Flujo Compresor 1 (SCFM)'] > 9000]
compresor_2 = compresor_2[compresor_2['Flujo Compresor 2 (SCFM)'] > 8000]
compresor_1 = compresor_1[compresor_1['Potencia Compresor 1 (kW)'] > 160]
compresor_2 = compresor_2[compresor_2['Potencia Compresor 2 (kW)'] > 150]



### Carga Compresor 1

In [ ]:
tabla = 'compresor_1'  # Nombre de la tabla en la base de datos
compresor_1.to_sql(tabla, engine, if_exists='append', index=True, index_label='Fecha', schema='public', method='multi', chunksize=1000)

print(f"Los datos se han subido correctamente a la tabla '{tabla}' en la base de datos '{db_name}'.")

### Carga Compresor 2

In [ ]:
tabla = 'compresor_2'  # Nombre de la tabla en la base de datos
compresor_2.to_sql(tabla, engine, if_exists='append', index=True, index_label='Fecha', schema='public', method='multi', chunksize=1000)

print(f"Los datos se han subido correctamente a la tabla '{tabla}' en la base de datos '{db_name}'.")

### Graficar los datos de los compresores

In [ ]:
fig = px.scatter(compresor_1, x='Flujo Compresor 1 (SCFM)', y = 'Potencia Compresor 1 (kW)', color = 'Temperatura Compresor 1 (°C)' )
fig.show()
fig = px.scatter(compresor_2, x='Flujo Compresor 2 (SCFM)', y = 'Potencia Compresor 2 (kW)', color = 'Temperatura Compresor 2 (°C)' )
fig.show()

In [ ]:
# KPI vs Flujo por temoperatura
fig = px.scatter(compresor_1, x='Flujo Compresor 1 (SCFM)', y = 'KPI Compresor 1', color = 'Temperatura Compresor 1 (°C)' )
fig.show()
fig = px.scatter(compresor_2, x='Flujo Compresor 2 (SCFM)', y = 'KPI Compresor 2', color = 'Temperatura Compresor 2 (°C)' )
fig.show()

In [ ]:
# KPI vs Flujo por presion

fig = px.scatter(compresor_1, x='Flujo Compresor 1 (SCFM)', y = 'KPI Compresor 1', color = 'Presion Compresor 1 (PSI)' )
fig.show()
fig = px.scatter(compresor_2, x='Flujo Compresor 2 (SCFM)', y = 'KPI Compresor 2', color = 'Presion Compresor 2 (PSI)' )
fig.show()

In [ ]:

compresor_1['Fechas'] = pd.cut(compresor_1.index, bins=10,)  # Crear una nueva columna 'Fechas' con los intervalos de tiempo
compresor_2['Fechas'] = pd.cut(compresor_2.index, bins=10,)  # Crear una nueva columna 'Fechas' con los intervalos de tiempo

In [ ]:
# kpi vs flujo por fechas
fig = px.scatter(compresor_1, x='Flujo Compresor 1 (SCFM)', y = 'KPI Compresor 1', color = 'Fechas' )
fig.show()
fig = px.scatter(compresor_2, x='Flujo Compresor 2 (SCFM)', y = 'KPI Compresor 2', color = 'Fechas' )
fig.show()

In [ ]:
# Presion vs Flujo por kpi

fig = px.scatter(compresor_1, x='Flujo Compresor 1 (SCFM)', color = 'KPI Compresor 1', y = 'Presion Compresor 1 (PSI)' )
fig.show()
fig = px.scatter(compresor_2, x='Flujo Compresor 2 (SCFM)', color = 'KPI Compresor 2', y = 'Presion Compresor 2 (PSI)' )
fig.show()

In [ ]:

fig = px.scatter(compresor_1, y='Potencia Compresor 1 (kW)', color = 'KPI Compresor 1', x = 'Temperatura Compresor 1 (°C)' )
fig.show()
fig = px.scatter(compresor_2, y='Potencia Compresor 2 (kW)', color = 'KPI Compresor 2', x = 'Temperatura Compresor 2 (°C)' )
fig.show()

In [ ]:
corr_comp_1 = compresor_1.corr()
corr_comp_2 = compresor_2.corr()

fig = px.imshow(corr_comp_1, text_auto=True, aspect="auto", title='Correlación Compresor 1')
fig.update_layout(xaxis_title='Variables', yaxis_title='Variables', width=800, height=600)
fig.show()

fig = px.imshow(corr_comp_2, text_auto=True, aspect="auto", title='Correlación Compresor 2')
fig.update_layout(xaxis_title='Variables', yaxis_title='Variables', width=800, height=600)
fig.show()

## Modelo Horario

In [ ]:
compresor_1 = compresores_horario[['Flujo Compresor 1 (SCFM)', 'Potencia Compresor 1 (kW)', 'KPI Compresor 1', 'Presion Compresor 1 (PSI)', 'Temperatura Compresor 1 (°C)']].dropna()
compresor_2 = compresores_horario[['Flujo Compresor 2 (SCFM)', 'Potencia Compresor 2 (kW)', 'KPI Compresor 2', 'Presion Compresor 2 (PSI)', 'Temperatura Compresor 2 (°C)']].dropna()
compresor_1 = compresor_1[compresor_1['Flujo Compresor 1 (SCFM)'] > 9000]
compresor_2 = compresor_2[compresor_2['Flujo Compresor 2 (SCFM)'] > 8000]
compresor_1 = compresor_1[compresor_1['Potencia Compresor 1 (kW)'] > 160]
compresor_2 = compresor_2[compresor_2['Potencia Compresor 2 (kW)'] > 150]

In [ ]:
compresor_1_scaled = compresor_1.copy()
compresor_2_scaled = compresor_2.copy()

compresor_1_scaled['Flujo Compresor 1 (SCFM)'] = compresor_1_scaled['Flujo Compresor 1 (SCFM)'] / 1000
compresor_2_scaled['Flujo Compresor 2 (SCFM)'] = compresor_2_scaled['Flujo Compresor 2 (SCFM)'] / 1000

In [ ]:
from scipy.optimize import curve_fit

def modelo(x, a, b, c):
    return a*x ** (-b) + c # Definición del modelo a ajustar

# Ajuste de curvas para los compresores

def calculate_lb(data, x, y):
    df = data[[x, y]].dropna() # Eliminar filas con valores NaN

    popt, _ = curve_fit(modelo, df[x], df[y], maxfev=1000000,) # Ajustar el modelo a los datos
    
    # Calcular los valores ajustados
    y_fit = modelo(df[x], *popt)
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df[x], y=df[y], mode='markers', name='Datos Reales', marker_color = 'blue')) # Datos reales
    fig.add_trace(go.Scatter(x=df[x], y=y_fit, mode='markers', name=f'Modelo Ajustado {popt[0]:.2f}x^(-{popt[1]:.2f}) + {popt[2]:.2f}', marker_color = 'red')) # Datos ajustados

    fig.update_layout(title=f'Ajuste: {x} vs {y}',
                  xaxis_title=x,
                  yaxis_title=y,
                  legend = dict(x=0.5, y=1),
                  width=800, height=600)
    print(f'Ajuste: {x} vs {y}')
    print(f'Parámetros del modelo: {popt}')
    print(f'Ecuación del modelo: {popt[0]:.4f}x^(-{popt[1]:.4f}) + {popt[2]:.4f}')
    fig.show()
    


In [96]:
calculate_lb(compresor_1_scaled, 'Flujo Compresor 1 (SCFM)', 'KPI Compresor 1')
calculate_lb(compresor_2_scaled, 'Flujo Compresor 2 (SCFM)', 'KPI Compresor 2')

Ajuste: Flujo Compresor 1 (SCFM) vs KPI Compresor 1
Parámetros del modelo: [ 4.13716243e+00  3.21877488e-03 -4.08951435e+00]
Ecuación del modelo: 4.1372x^(-0.0032) + -4.0895


Ajuste: Flujo Compresor 2 (SCFM) vs KPI Compresor 2
Parámetros del modelo: [ 2.32409732e+01  6.51608139e-04 -2.31891738e+01]
Ecuación del modelo: 23.2410x^(-0.0007) + -23.1892
